In [1]:
import sys, os
from pathlib import Path
sys.path.append(os.path.dirname(Path().resolve()))
os.path.dirname(Path().resolve()), sys.path

('/home/umenty/dev/hackathon/Noimyu',
 ['/home/umenty/dev/hackathon/Noimyu/suzuki_test',
  '/home/umenty/miniconda3/envs/geek2/lib/python39.zip',
  '/home/umenty/miniconda3/envs/geek2/lib/python3.9',
  '/home/umenty/miniconda3/envs/geek2/lib/python3.9/lib-dynload',
  '',
  '/home/umenty/miniconda3/envs/geek2/lib/python3.9/site-packages',
  '/home/umenty/dev/hackathon/Noimyu'])

In [2]:
from soundAI.rl_model import RLModel
model = RLModel([["delightful", "depressing", "hyper"], ["jazz", "rock"]])
model

In [3]:
all_inputs = model.network.generate_all_input_combinations(debug=True)
all_inputs

category_onehot_list
[((1, 0, 0), (0, 1, 0), (0, 0, 1)), ((1, 0), (0, 1))]
all_input_combinations
tensor([[1., 0., 0., 1., 0.],
        [1., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.],
        [0., 1., 0., 0., 1.],
        [0., 0., 1., 1., 0.],
        [0., 0., 1., 0., 1.]])


tensor([[1., 0., 0., 1., 0.],
        [1., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.],
        [0., 1., 0., 0., 1.],
        [0., 0., 1., 1., 0.],
        [0., 0., 1., 0., 1.]])

In [4]:
for input in all_inputs:
    print(
        model.catman.prompt_to_action_id(
            model.catman.combination_tensor_to_text(
                input
            ), model.network
        )
    )

0
1
2
3
4
5
